In [1]:
#import relevant packages for analysis

from __future__ import print_function

import os
import sys


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_context('talk')
import random
from scipy.stats import randint
import re
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

PROJ_ROOT = os.path.join(os.pardir)

In [2]:
%load_ext watermark
%watermark -a "Bryan Dickinson" -d -t -v -p numpy,pandas,scikitlearn

Bryan Dickinson 2019-08-07 13:34:13 

CPython 3.7.3
IPython 7.7.0

numpy 1.16.4
pandas 0.25.0
scikitlearn not installed


In [3]:
#Create the path to the data and read into a pandas dataframe

terry_data = os.path.join(PROJ_ROOT, 
                         'data', 'processed',
                         'Terry_Stops_Clean.csv')

data = pd.read_csv(terry_data, parse_dates = ['date'], 
                   index_col = 'date', dtype = {'officer_race':'category','officer_gender':'category',
                                                'subject_age':'category','subject_race':'category',
                                                'subject_gender': 'category','stop_resolution': 'category',
                                                'weapon_type':'category','initial_call_type':'category',
                                                'call_type':'category','arrest':'int32', 'frisk':'float',
                                                'precinct':'category', 'sector':'category', 'beat': 'category'})

data.sort_index(inplace = True)

In [4]:
data.head()

,officer_id,officer_age,officer_race,officer_gender,officer_squad,subject_id,subject_age,subject_race,subject_gender,stop_resolution,weapon_type,initial_call_type,call_type,arrest,frisk,precinct,sector,beat
date,,,,,,,,,,,,,,,,,,
2015-03-15,115,60.0,Hispanic or Latino,M,NaN,10305.0,1 - 17,Black,Female,GO for Prosecutorial Referral,Lethal Cutting Instrument,NaN,NaN,0,1.0,NaN,NaN,NaN
2015-03-16,1757,31.0,White,M,NaN,1432.0,18 - 25,Black,Male,Arrest with GO or Supplemental,None,NaN,NaN,1,0.0,NaN,NaN,NaN
2015-03-16,1735,38.0,White,M,NaN,20151.0,36 - 45,Multi-Racial,Male,Street Check,None,NaN,NaN,0,0.0,NaN,NaN,NaN
2015-03-16,1735,38.0,White,M,NaN,22667.0,18 - 25,White,Male,Street Check,None,NaN,NaN,0,0.0,NaN,NaN,NaN
2015-03-17,1735,38.0,White,M,NaN,10743.0,26 - 35,White,Male,Street Check,None,NaN,NaN,0,0.0,NaN,NaN,NaN


In [5]:
def split_mean(x):
    #Function to split the Age bins and return the mean of the two numbers
        if '-' in x:
            split_list = x.split('-')
            mean = (float(split_list[0]) + float(split_list[1]))/2
        else:
            mean = 56
        return mean

In [6]:
ofc_columns = ['officer_id','subject_race', 'call_type', 'initial_call_type', 'beat']

df = data[ofc_columns]

# view only stops when the officer initiates the stop
df = df[df.call_type == 'ONVIEW']


#remove ambiguous subject_races
df = df[(df.subject_race != 'Unknown') & (df.subject_race != 'Other')]

#subset the data to include the initial call types that prompted the stop, limit to 5 or more to capture any trends
ls = list(df.initial_call_type.value_counts()[df.initial_call_type.value_counts() >= 5].index)
df = df[df.initial_call_type.isin(ls)]

#subset the data to include officers with at least 15 stops or more
ls = list(df.officer_id.value_counts()[df.officer_id.value_counts() >= 15].index)
df = df[df.officer_id.isin(ls)]
df.head()

df.drop(['call_type'], axis = 1, inplace = True) #drop the call type feature

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2269 entries, 2015-03-18 to 2019-05-06
Data columns (total 4 columns):
officer_id           2269 non-null int64
subject_race         2269 non-null category
initial_call_type    2269 non-null category
beat                 2269 non-null category
dtypes: category(3), int64(1)
memory usage: 53.9 KB


In [7]:

#create the pattern to match the beat entries
pattern = re.compile( '^[A-Z][1-9]$')

#drop any NaNs in the beat column
df = df.dropna(subset = ['beat'])

#use the pattern created to subset the data and get rid of the erroneous entries
df = df[(df.beat.str.contains(pattern))]

#remove all unused categories
for col in df.select_dtypes(include = ['category']).columns:
    df[col] = df[col].cat.remove_unused_categories()

    #capture the category codes & corresponding strings for the 'race' classes
race_cat_codes = dict(enumerate(df['subject_race'].cat.categories))

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2253 entries, 2015-03-18 to 2019-05-06
Data columns (total 4 columns):
officer_id           2253 non-null int64
subject_race         2253 non-null category
initial_call_type    2253 non-null category
beat                 2253 non-null category
dtypes: category(3), int64(1)
memory usage: 47.8 KB


In [8]:
df.head()

,officer_id,subject_race,initial_call_type,beat
date,,,,
2015-03-18,1735,White,WARRANT - FELONY PICKUP,E2
2015-03-18,1735,White,WARRANT - FELONY PICKUP,E2
2015-03-18,1735,White,WARRANT - FELONY PICKUP,E2
2015-03-18,1827,American Indian / Alaskan Native,TRAFFIC STOP - OFFICER INITIATED ONVIEW,C2
2015-03-18,1735,Black,WARRANT - FELONY PICKUP,E2


In [9]:
#import relevent classifiers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#import preprocessing, metrics & pipelines
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, roc_auc_score


In [10]:
#The metric that will be used is log loss. log loss is a log function is a measure of error. 
#The error should be as small as possible.

def compute_log_loss(predicted, actual, eps = 1e-14):
    #computes the logarithmic loss between predicted and actual when these are 1d arrays
    predicted = np.clip(predicted, eps, 1-eps)
    loss = -1 * np.mean(actual * np.log(predicted)
                       + (1 - actual)
                       * np.log(1-predicted))
    return loss

def consolidate_array(arr, cols = [0,1,2,3,4,5]):
    #function to transform the dummies array to a single column
    
    df = pd.DataFrame(arr, columns = cols)
    return(df.idxmax(axis = 1).values)

In [11]:

#set up the target variable
y = pd.get_dummies(df['subject_race']).values


#set the x variables by converting the categorical text features to dummy variables
X = pd.get_dummies(df.reset_index(drop = True).drop(['subject_race'], axis = 1),
                          columns = ['officer_id', 'beat', 'initial_call_type'] )

In [12]:
print(y.shape)
print(X.shape)

(2253, 6)
(2253, 190)


In [13]:
#split the data to test & training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 5, stratify = y)


#build the pipline with upsampling & scaling the data
pipeline = imbPipeline([('sm', SMOTE(random_state = 5, sampling_strategy = 'not majority')),
                        ('scale', StandardScaler()),
                        ('clf', OneVsRestClassifier(LogisticRegression()))
                       ])

#paramters for tuning
parameters = [
    {'clf' : [ OneVsRestClassifier(LogisticRegression(random_state = 5))],
    'clf__estimator__C' : np.logspace(-5, 8, 10),
    'clf__estimator__solver' : ['lbfgs']},
    {'clf' : [ OneVsRestClassifier(RandomForestClassifier(random_state = 5))],
    'clf__estimator__max_depth':[4, 3, None],
     'clf__estimator__criterion': ['gini'],
    'clf__estimator__n_estimators' : [10,25,50,100],
    'clf__estimator__max_features' : ['auto', 2, 4]},
]

#create the grid search object

cv = GridSearchCV(pipeline, param_grid = parameters,
                    scoring = 'neg_log_loss', 
                    refit = True, 
                    cv = 5, 
                    verbose= True, 
                    n_jobs = -1)


#fit the data
cv.fit(X_train, y_train)

Fitting 5 folds for each of 46 candidates, totalling 230 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 230 out of 230 | elapsed:  3.7min finished
C:\Users\Bryan\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('sm',
                                        SMOTE(k_neighbors=5, kind='deprecated',
                                              m_neighbors='deprecated',
                                              n_jobs=1, out_step='deprecated',
                                              random_state=5, ratio=None,
                                              sampling_strategy='not majority',
                                              svm_estimator='deprecated')),
                                       ('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        One...
                                            

In [14]:
fname = 'predict_race_model.sav'
pickle.dump(cv, open(fname, 'wb'))

In [15]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('sm',
                 SMOTE(k_neighbors=5, kind='deprecated',
                       m_neighbors='deprecated', n_jobs=1,
                       out_step='deprecated', random_state=5, ratio=None,
                       sampling_strategy='not majority',
                       svm_estimator='deprecated')),
                ('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=0.21544346900318823,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                       

In [16]:
#predict on the test set
y_pred = cv.predict(X_test)

y_predict_proba = cv.predict_proba(X_test)

y_t = consolidate_array(y_test)
y_p = consolidate_array(y_pred)

In [17]:
#accuracy score
acc_score = cv.score(X_test, y_test)
log_loss = compute_log_loss(y_predict_proba, y_test)
print('The accuracy score is: {}'.format(acc_score))
print('The Log Loss score is: {}'.format(log_loss))


#print the confusion matrix and classification report from the best model
print(confusion_matrix(y_t, y_p))
print(classification_report(y_t, y_p))

The accuracy score is: -1.3663228770811118
The Log Loss score is: 0.3626715342134398
[[ 11   0   2   0   0   1]
 [ 12   0   1   0   0   0]
 [119   0   9   0   0  12]
 [ 22   0   0   0   0   3]
 [ 15   0   1   0   0   0]
 [208   0   4   0   0  31]]
              precision    recall  f1-score   support

           0       0.03      0.79      0.05        14
           1       0.00      0.00      0.00        13
           2       0.53      0.06      0.11       140
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        16
           5       0.66      0.13      0.21       243

    accuracy                           0.11       451
   macro avg       0.20      0.16      0.06       451
weighted avg       0.52      0.11      0.15       451



C:\Users\Bryan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
#print the log loss for each column
for col in np.arange(0,6):
    print( compute_log_loss(y_predict_proba[:,col], y_test[:,col]), '\t',race_cat_codes[col])

0.17736506860828938 	 American Indian / Alaskan Native
0.16231245721799537 	 Asian
0.6361940176404637 	 Black
0.25306143445002266 	 Hispanic
0.20350791630965773 	 Multi-Racial
0.74358831105421 	 White


In [19]:
pd.concat([pd.DataFrame(y_predict_proba), pd.DataFrame(y_test)], axis = 1).head(10)

,0,1,2,3,4,5,0,1,2,3,4,5
0,0.002959,0.002958,0.168457,0.003289,0.003262,0.829221,0,0,0,1,0,0
1,0.089833,0.093844,0.237074,0.160303,0.143212,0.358930,0,0,0,0,0,1
2,0.089738,0.101733,0.250687,0.142971,0.154033,0.356270,0,0,0,0,0,1
3,0.048963,0.048972,0.276105,0.049546,0.063643,0.369992,0,0,0,0,0,1
4,0.084836,0.090499,0.258997,0.142892,0.129215,0.350853,0,0,0,1,0,0
5,0.098147,0.096516,0.254551,0.132966,0.151957,0.347619,1,0,0,0,0,0
6,0.002787,0.002892,0.199449,0.003024,0.003213,0.821889,0,0,0,0,0,1
7,0.091648,0.106616,0.257991,0.129430,0.149395,0.350982,0,0,0,0,0,1
8,0.095650,0.100934,0.240881,0.139235,0.151757,0.351420,0,0,0,0,0,1
9,0.094725,0.104996,0.261266,0.136597,0.152919,0.355337,1,0,0,0,0,0


In [20]:

#obtain the feature importances from within the GridSearchCV, pipeline, & OneVsRest objects
feat_imp = [x.coef_ for x in cv.best_estimator_.steps[2][1].estimators_]
feat_imp = np.mean(feat_imp, axis = 0)

#place the feature importances in a dataframe
feature_importances = pd.DataFrame(feat_imp[0],
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance',ascending=True)
feature_importances.head(3)

,importance
beat_E3,-0.376671
beat_Q2,-0.330176
beat_D3,-0.307895


## Findings

**The model performance metrics:**
 - Log Loss .36
 - Precision .52
 - Recall .13
 - F1 Score .15
    
    The model is a small improvement from the previous model increasing Recall & the F1 Score, though is still not making predictions confidently enough to improve the F1 score to an acceptable level. We can see from predicted probabilities, the predictions are a bit more confident than the previous model. 
    The top 3 features are 'beat' locations and negative, suggesting that subjects stopped in these neighborhoods have a slightly higher probability of *not* being frisked.

A Stop does not necessarily lead to a 'frisk', another set of behaviors observed by an experienced officer needs to be evaluated before a frisk is legitimately performed.